In [ ]:
import torch
from openmmtools.constants import kB
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
from openmmtools.testsystems import AlanineDipeptideVacuum
import numpy as np
import mdtraj

import boltzgen.internal as ics

In [ ]:
# Generate a trajectory for use as training data
temperature = 298
kT = kB * temperature
testsystem = AlanineDipeptideVacuum()
vacuum_sim = app.Simulation(testsystem.topology,
                            testsystem.system,
                            mm.LangevinIntegrator(temperature * unit.kelvin , 1.0 / unit.picosecond, 1.0 * unit.femtosecond),
                            platform=mm.Platform.getPlatformByName('Reference')
                            )
vacuum_sim.context.setPositions(testsystem.positions)
vacuum_sim.minimizeEnergy()
vacuum_sim.reporters.append(mdtraj.reporters.HDF5Reporter('/home/vstimper/Documents/MPI-IS/Projects/bootstrap-flow/datasets/aldp.h5', 100))
vacuum_sim.step(100000)
del(vacuum_sim)

In [ ]:
aldp_traj = mdtraj.load('/home/vstimper/Documents/MPI-IS/Projects/bootstrap-flow/datasets/aldp.h5')

In [ ]:
z = [
    (0, [1, 4, 6]),
    (1, [4, 6, 8]),
    (2, [1, 4, 0]),
    (3, [1, 4, 0]),
    (4, [6, 8, 14]),
    (5, [4, 6, 8]),
    (7, [6, 8, 4]),
    (9, [8, 6, 4]),
    (10, [8, 6, 4]),
    (11, [10, 8, 6]),
    (12, [10, 8, 11]),
    (13, [10, 8, 11]),
    (15, [14, 8, 16]),
    (16, [14, 8, 6]),
    (17, [16, 14, 15]),
    (18, [16, 14, 8]),
    (19, [18, 16, 14]),
    (20, [18, 16, 19]),
    (21, [18, 16, 19])
]

backbone_indices = [8, 6, 14]

# center everything
aldp_traj.center_coordinates()

# superpose on the backbone
ind = aldp_traj.top.select("backbone")

aldp_traj.superpose(aldp_traj, 0, atom_indices=ind, ref_atom_indices=ind)

# Gather the training data into a pytorch Tensor with the right shape
training_data = aldp_traj.xyz
n_atoms = training_data.shape[1]
n_dim = n_atoms * 3
training_data_npy = training_data.reshape(-1, n_dim)
training_data = torch.from_numpy(training_data_npy.astype("float64"))

ic_transform = ics.InternalCoordinateTransform(n_dim, z, backbone_indices, training_data)

transformed_c, jac = ic_transform.forward(training_data)
back_trans, back_jac = ic_transform.inverse(transformed_c)


cic_transform = ics.CompleteInternalCoordinateTransform(n_dim, z, backbone_indices, training_data)

z, j = cic_transform.forward(training_data)
x, bj = cic_transform.inverse(z)
z_, _ = cic_transform.forward(x)
x_, _ = cic_transform.inverse(z_)

In [ ]:
print(torch.max(torch.abs(z_ - z)))
print(torch.max(torch.abs(x_ - x)))